In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.utils import set_random_seed as set_seed
set_seed(42)


In [2]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)


In [3]:
def WindowSplit(n, df):
    ##last day is considered counted by n the number of days (n-1 train, 1 test)
    indices = list(range(len(df.index)))
    result = []
    
    freq = n*96
    
    total_days = len(df.index)//96
    windows = total_days - n + 1
    
    
    start = 0
    for i in range(windows):
        folds = []
        for j in range(5):
            end = start+(n-2)*96 + 91 + 20*j +1
            train = indices[start:end]
            test = [indices[end]]
            folds.append((train,test))
            
        result.append(folds)
        start += 96
        
    return result, windows
        
            

In [4]:

def model1(ws, windows, scaler, df):
    
    mse1 = []
    
    for window in ws:
        
        for train_indx, val_indx in window:
            train = df.iloc[train_indx]
            test = df.iloc[val_indx]
        
        
            scaler.fit(train['310_PV_1'].to_frame())
            scaled_train = scaler.transform(train['310_PV_1'].to_frame())
            scaled_test = scaler.transform(test['310_PV_1'].to_frame())
        
            n_input = 3 ##check in book if this is determined with ACF PACF or something else
            n_features = 1 ## see if we should make the neural network with multiple time series (for now just 1)
            generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
     
            mod = Sequential()
            mod.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
            mod.add(Dense(1))
            mod.compile(optimizer='adam', loss='mse')
       
        
            print(mod.summary())
        
            mod.fit(generator,epochs=11,verbose = 0)#keep low for now, edit later
        
         
            last_batch = scaled_train[-3:].reshape((1, n_input, n_features))
    
            pred = mod.predict(last_batch)
            pred = scaler.inverse_transform(pred)
    
            error = mse([test['310_PV_1'][0]],pred[0])
            
        
        
            mse1.append(error)

    print(mse1)

    
    return mse1

    

In [5]:
def modelNaive(ws,df):

    mse3 = []
    
    
    for window in ws:
        
        for train_indx, val_indx in window:
            train = df.iloc[train_indx]
            test = df.iloc[val_indx]
    
            pred = train['310_PV_1'][-1]
    
    
            error = mse([test['310_PV_1'][0]],[pred])
            
            mse3.append(error)
            
    print(mse3)
    return mse3

In [6]:
def runModels(df):
    ws, windows =  WindowSplit(5,df)#window split
    scaler = MinMaxScaler()
    mse1 = model1(ws,windows,scaler,df)
    mse2 = modelNaive(ws,df)
    
    return mse1, mse2

    

In [7]:


df1 = df[df.index.month.isin([1,2,3])]
ws = WindowSplit(5,df1)
print(ws)
df2 = df[df.index.month.isin([4,5,6])]
df3 = df[df.index.month.isin([7,8,9])]
df4 = df[df.index.month.isin([10,11,12])]

([[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

In [8]:
mse1, mse2 = runModels(df1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 159ms/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                      

 lstm_9 (LSTM)               (None, 100)               40800     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 100)               40800     
                                                                 
 dense_10 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_____________________________________________________

 lstm_20 (LSTM)              (None, 100)               40800     
                                                                 
 dense_20 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 134ms/step
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 100)               40800     
                                                                 
 dense_21 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_____________________________________________________

 lstm_31 (LSTM)              (None, 100)               40800     
                                                                 
 dense_31 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 141ms/step
Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 100)               40800     
                                                                 
 dense_32 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_____________________________________________________

 lstm_42 (LSTM)              (None, 100)               40800     
                                                                 
 dense_42 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_43 (LSTM)              (None, 100)               40800     
                                                                 
 dense_43 (Dense)            (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_____________________________________________________

...

 lstm_383 (LSTM)             (None, 100)               40800     
                                                                 
 dense_383 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_384"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_384 (LSTM)             (None, 100)               40800     
                                                                 
 dense_384 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
____________________________________________________

 lstm_394 (LSTM)             (None, 100)               40800     
                                                                 
 dense_394 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 127ms/step
Model: "sequential_395"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_395 (LSTM)             (None, 100)               40800     
                                                                 
 dense_395 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
____________________________________________________

 lstm_405 (LSTM)             (None, 100)               40800     
                                                                 
 dense_405 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_406"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_406 (LSTM)             (None, 100)               40800     
                                                                 
 dense_406 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
____________________________________________________

 lstm_416 (LSTM)             (None, 100)               40800     
                                                                 
 dense_416 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_417"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_417 (LSTM)             (None, 100)               40800     
                                                                 
 dense_417 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
____________________________________________________

 lstm_427 (LSTM)             (None, 100)               40800     
                                                                 
 dense_427 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_428"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_428 (LSTM)             (None, 100)               40800     
                                                                 
 dense_428 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
____________________________________________________

In [9]:
mse3, mse4 = runModels(df2)

Model: "sequential_435"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_435 (LSTM)             (None, 100)               40800     
                                                                 
 dense_435 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 127ms/step
Model: "sequential_436"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_436 (LSTM)             (None, 100)               40800     
                                                                 
 dense_436 (Dense)           (None, 1)                 101       
                                

1/1 [==============================] - 0s 137ms/step
Model: "sequential_446"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_446 (LSTM)             (None, 100)               40800     
                                                                 
 dense_446 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 126ms/step
Model: "sequential_447"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_447 (LSTM)             (None, 100)               40800     
                                                                 
 dense_447 (Dense)           (None, 1)       

None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_457"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_457 (LSTM)             (None, 100)               40800     
                                                                 
 dense_457 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_458"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_458 (LSTM)             (None, 100)               40800     
                                                                 
 dense_458 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_468"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_468 (LSTM)             (None, 100)               40800     
                                                                 
 dense_468 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_469"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_469 (LSTM)             (None, 100)               40800     
                                                                 
 dense_469 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_479"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_479 (LSTM)             (None, 100)               40800     
                                                                 
 dense_479 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_480"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_480 (LSTM)             (None, 100)               40800     
                                                                 
 dense_480 (Dense)           (None, 1)  

...

None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_820"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_820 (LSTM)             (None, 100)               40800     
                                                                 
 dense_820 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_821"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_821 (LSTM)             (None, 100)               40800     
                                                                 
 dense_821 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 127ms/step
Model: "sequential_831"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_831 (LSTM)             (None, 100)               40800     
                                                                 
 dense_831 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 132ms/step
Model: "sequential_832"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_832 (LSTM)             (None, 100)               40800     
                                                                 
 dense_832 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_842"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_842 (LSTM)             (None, 100)               40800     
                                                                 
 dense_842 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_843"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_843 (LSTM)             (None, 100)               40800     
                                                                 
 dense_843 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_853"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_853 (LSTM)             (None, 100)               40800     
                                                                 
 dense_853 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_854"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_854 (LSTM)             (None, 100)               40800     
                                                                 
 dense_854 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_864"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_864 (LSTM)             (None, 100)               40800     
                                                                 
 dense_864 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 158ms/step
Model: "sequential_865"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_865 (LSTM)             (None, 100)               40800     
                                                                 
 dense_865 (Dense)           (None, 1)  

In [10]:
mse5, mse6 = runModels(df3)

Model: "sequential_870"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_870 (LSTM)             (None, 100)               40800     
                                                                 
 dense_870 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_871"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_871 (LSTM)             (None, 100)               40800     
                                                                 
 dense_871 (Dense)           (None, 1)                 101       
                                

1/1 [==============================] - 0s 162ms/step
Model: "sequential_881"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_881 (LSTM)             (None, 100)               40800     
                                                                 
 dense_881 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 127ms/step
Model: "sequential_882"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_882 (LSTM)             (None, 100)               40800     
                                                                 
 dense_882 (Dense)           (None, 1)       

None
1/1 [==============================] - 0s 127ms/step
Model: "sequential_892"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_892 (LSTM)             (None, 100)               40800     
                                                                 
 dense_892 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_893"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_893 (LSTM)             (None, 100)               40800     
                                                                 
 dense_893 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_903"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_903 (LSTM)             (None, 100)               40800     
                                                                 
 dense_903 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_904"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_904 (LSTM)             (None, 100)               40800     
                                                                 
 dense_904 (Dense)           (None, 1)  

None
1/1 [==============================] - 0s 151ms/step
Model: "sequential_914"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_914 (LSTM)             (None, 100)               40800     
                                                                 
 dense_914 (Dense)           (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_915"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_915 (LSTM)             (None, 100)               40800     
                                                                 
 dense_915 (Dense)           (None, 1)  

...

None
1/1 [==============================] - 0s 165ms/step
Model: "sequential_1277"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1277 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1277 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1278"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1278 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1278 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 132ms/step
Model: "sequential_1288"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1288 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1288 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 138ms/step
Model: "sequential_1289"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1289 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1289 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_1299"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1299 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1299 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1300"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1300 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1300 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 127ms/step
[0.5986781161624016, 0.23650037263864407, 0.06571227396901246, 3.1311760330103535, 7.9620986569042085, 0.018539678635647726, 0.3454665432518085, 1.2734457145857891, 0.01604071486121144, 2.0682548175922135, 0.8160866747070513, 1.3305861311328089, 2.4124934224679637, 45.74265156294922, 5.188696929306527, 0.004650954907172818, 0.2003121524573939, 0.3958591690671993, 0.0015357008825196002, 22.84917004391344, 0.0005997975383054217, 0.4585543424733592, 0.014677528629954048, 0.02641980464373458, 0.6511243538930157, 0.41936452241967165, 0.07654206631784931, 0.043664946736877694, 0.04548757820531929, 1.1457079138059976, 0.012351244613491645, 0.09756295533500633, 0.0291138995628409, 63.09720931238251, 3.498829663570076, 0.02698397269206922, 0.15126334351503232, 2.3935782091741853, 0.8484540904175367, 1.9631154059234044, 0.108684139066232, 0.06787925340562076, 0.36435355681287707, 1.1015372719649306, 0.15215926693046508, 0.7145642230317801

[2.0126666212905243e-09, 1.5067221281049196e-09, 0.007975410085405575, 3.359065532169197, 16.77203151394886, 1.6350504250896649e-10, 1.7239029350380383e-10, 0.18098494682878533, 0.045773863126081414, 0.009511441094463445, 2.189391987270168e-10, 2.2839086339847222e-10, 0.09313214072257665, 54.958559898337555, 8.08059832471042, 2.803480596374639e-10, 1.144335786661275e-10, 0.19713605565303952, 0.8089959651968283, 35.149488582374545, 1.635050425543947e-10, 2.2635048342257126e-10, 0.07540786131902076, 0.20497945599816061, 0.009516937454100394, 1.789736406525873e-10, 1.7061822337892378e-10, 0.1076690138233109, 0.6187774828074099, 0.0005786679295272246, 1.3067003845603037e-10, 3.054572046026423e-10, 0.15160233513141633, 19.955844230496645, 0.01904246236960442, 2.3678402451558517e-10, 2.2635048348270298e-10, 1.5843972220584235, 0.00037687373006475676, 0.009516937454100049, 1.6805174053870345e-09, 1.6547080088894625e-09, 0.008981091205303346, 1.3301433037084245, 0.0005799973270636277, 1.524676

In [11]:
mse7, mse8 = runModels(df4)

Model: "sequential_1310"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1310 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1310 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1311"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1311 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1311 (Dense)          (None, 1)                 101       
                              

1/1 [==============================] - 0s 130ms/step
Model: "sequential_1321"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1321 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1321 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1322"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1322 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1322 (Dense)          (None, 1)     

None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_1332"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1332 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1332 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1333"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1333 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1333 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_1343"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1343 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1343 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1344"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1344 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1344 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 130ms/step
Model: "sequential_1354"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1354 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1354 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 129ms/step
Model: "sequential_1355"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1355 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1355 (Dense)          (None, 1)

...

None
1/1 [==============================] - 0s 140ms/step
Model: "sequential_1717"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1717 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1717 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_1718"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1718 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1718 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_1728"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1728 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1728 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 131ms/step
Model: "sequential_1729"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1729 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1729 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 132ms/step
Model: "sequential_1739"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1739 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1739 (Dense)          (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 128ms/step
Model: "sequential_1740"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1740 (LSTM)            (None, 100)               40800     
                                                                 
 dense_1740 (Dense)          (None, 1)

None
1/1 [==============================] - 0s 129ms/step
[1.6103821780620473, 0.25934967010543236, 15.436444386219215, 4.662344681664606, 0.001535372190099571, 1.04392747867541, 0.15932877412540822, 0.24947367388137123, 1.0756908588820193, 0.0002956065768759936, 3.7593604081652847, 0.11058403326218719, 0.21416248259054352, 1.627418304208465, 0.6303718731775468, 0.010269255969728391, 0.4209285153392078, 0.2554504312131282, 1.5873641995180825, 2.5367510051369164, 0.2895081757821359, 0.2533984616627141, 1.183660268291761, 13.794265132960218, 0.04933814211188911, 0.053345600828318206, 2.5069006949664243, 0.7293832987489531, 8.368286947367261, 0.26122316857735206, 0.07640397610484742, 0.008741662149591076, 161.5163269182617, 4.072410224056725, 2.4957014338566523, 0.009820407731660746, 0.06921607276653831, 15.376099550804453, 1.2005475883693122, 3.207631135939567, 0.40426983457233967, 0.10717167057291971, 7.076836524040082, 0.9332656706620227, 0.9640304545841476, 0.34252189488117984, 0.5466

[1.541152848023584e-07, 1.5386726776139646e-07, 0.05708099606845631, 4.464755044718751, 0.0005970552847082552, 1.525903851325814e-07, 1.5735060132316297e-07, 0.0005324083585203533, 0.11101073493510158, 0.019147260332606938, 1.556972287095201e-07, 1.5542678776489563e-07, 0.39064485563629153, 0.0028223470531493116, 0.009591066231718208, 1.5411528480244555e-07, 1.5386726776157066e-07, 0.11486425509019021, 0.6193772972528719, 0.0005971515213000061, 1.54132697430524e-07, 1.5891672587317466e-07, 0.08395078099518818, 5.290683715967004, 0.019147805301700984, 1.572551238718337e-07, 1.5698333270491361e-07, 0.030620129899194687, 0.1710144475430935, 0.009591451934945474, 1.5566526510162166e-07, 1.554160034932793e-07, 154.4489303528031, 1.3916010074203298, 0.0005971515213001906, 1.5413269743078553e-07, 1.589167258745024e-07, 1.1875437142324508, 0.44424920246560823, 0.01914840904866521, 1.5899007551659822e-07, 1.5871678851508614e-07, 10.080550491824123, 0.2701884884867032, 0.009591879241003812, 1.57

In [12]:
print('season 1')

print('model 1:')
print(mse1)
print('model Naive:')
print(mse2)


season 1
model 1:
[1.7029779871109452, 0.6886450388281968, 1.355115243149221, 0.041008105400063284, 0.9887779650741223, 0.003307807927168101, 0.16464928731414827, 0.04594315312170753, 0.42488960542529935, 0.0005545900506805718, 0.1339316808041553, 1.2112124957248367, 2.3857372587281542, 0.1747250121197346, 0.05811255061532017, 0.004733745088389826, 0.9974927599744621, 1.5182637209209162, 20.1606427946587, 0.08004981693664272, 0.25631228285575425, 0.9427823539032775, 7.645633120419615, 0.3926486676014007, 0.012279928563951352, 0.029604293363191505, 0.19205489858643363, 1.1457147402095327, 0.045655690186943675, 0.31868595732083277, 1.4839886805766418, 0.40972436174237975, 64.18370077554349, 256.52813835607856, 0.13672492329845468, 0.1282613414421129, 0.7146815067107631, 0.3581752593644608, 2.730346864596878, 2.8869853217927126, 0.00822800773634034, 0.14761617217903378, 0.3344018746945383, 10.245837008787165, 0.20247925205748651, 0.8789329179889533, 0.00017935698013760764, 6.6523016424437

In [13]:
print('season 1')

print('model 1:')
print(mse3)
print('model Naive:')
print(mse4)


season 1
model 1:
[1.3942113423974982, 0.008122121378400245, 0.5223210909967679, 14.208346306086703, 6.446174539914731, 0.16588037219536242, 0.27950730176512717, 1.9465480426457578, 2.323450888425698, 1.0875181590133256, 1.412011198588145, 4.123131849493838, 47.16260047327319, 7.28995410228189, 0.9942870499303152, 0.11067883679549559, 5.2734068118303155, 109.51224562428177, 44.81697197973089, 1.5171383601176545, 1.137211902262402, 0.585080925412936, 0.8118314792569437, 8.966101705081305, 0.006841853720186484, 1.5025376800998673, 1.2876004550926936, 1.0523936338107016, 0.5266002547263366, 0.07978253270115643, 0.6670394966776745, 1.959413389684733, 0.14428996464427177, 2.2590894845298477, 0.35261646312519196, 0.32967373675509737, 0.12358721421455318, 3.6687599330091523, 1.6200395112750074, 0.24229748836235712, 0.030405719843659185, 0.04128311815267023, 0.2804687797149602, 0.46968046253669055, 0.14290516756089383, 0.2018940479087798, 0.42367913698328247, 30.853035166592512, 53.95909531628

In [14]:
print('season 1')

print('model 1:')
print(mse5)
print('model Naive:')
print(mse6)


season 1
model 1:
[0.5986781161624016, 0.23650037263864407, 0.06571227396901246, 3.1311760330103535, 7.9620986569042085, 0.018539678635647726, 0.3454665432518085, 1.2734457145857891, 0.01604071486121144, 2.0682548175922135, 0.8160866747070513, 1.3305861311328089, 2.4124934224679637, 45.74265156294922, 5.188696929306527, 0.004650954907172818, 0.2003121524573939, 0.3958591690671993, 0.0015357008825196002, 22.84917004391344, 0.0005997975383054217, 0.4585543424733592, 0.014677528629954048, 0.02641980464373458, 0.6511243538930157, 0.41936452241967165, 0.07654206631784931, 0.043664946736877694, 0.04548757820531929, 1.1457079138059976, 0.012351244613491645, 0.09756295533500633, 0.0291138995628409, 63.09720931238251, 3.498829663570076, 0.02698397269206922, 0.15126334351503232, 2.3935782091741853, 0.8484540904175367, 1.9631154059234044, 0.108684139066232, 0.06787925340562076, 0.36435355681287707, 1.1015372719649306, 0.15215926693046508, 0.7145642230317801, 0.6736890941431537, 10.523706402993543

In [15]:
print('season 1')

print('model 1:')
print(mse7)
print('model Naive:')
print(mse8)


season 1
model 1:
[1.6103821780620473, 0.25934967010543236, 15.436444386219215, 4.662344681664606, 0.001535372190099571, 1.04392747867541, 0.15932877412540822, 0.24947367388137123, 1.0756908588820193, 0.0002956065768759936, 3.7593604081652847, 0.11058403326218719, 0.21416248259054352, 1.627418304208465, 0.6303718731775468, 0.010269255969728391, 0.4209285153392078, 0.2554504312131282, 1.5873641995180825, 2.5367510051369164, 0.2895081757821359, 0.2533984616627141, 1.183660268291761, 13.794265132960218, 0.04933814211188911, 0.053345600828318206, 2.5069006949664243, 0.7293832987489531, 8.368286947367261, 0.26122316857735206, 0.07640397610484742, 0.008741662149591076, 161.5163269182617, 4.072410224056725, 2.4957014338566523, 0.009820407731660746, 0.06921607276653831, 15.376099550804453, 1.2005475883693122, 3.207631135939567, 0.40426983457233967, 0.10717167057291971, 7.076836524040082, 0.9332656706620227, 0.9640304545841476, 0.34252189488117984, 0.5466418274272319, 0.03254588017811905, 1.091

In [20]:
d = {'Season 1':mse2,'Season 2':mse4,'Season 3':mse6,'Season 4':mse8}
df_errors_Naive = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
d = {'Season 1':mse1,'Season 2':mse3,'Season 3':mse5,'Season 4':mse7}
df_errors_LSTM = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
df_errors_Naive.to_csv('Naive_errors.csv')
df_errors_LSTM.to_csv('LSTM_errors.csv')



{'Season 1': 0       1.702978
 1       0.688645
 2       1.355115
 3       0.041008
 4       0.988778
          ...    
 430     2.449966
 431     0.806584
 432    12.131042
 433     8.323100
 434     0.663528
 Length: 435, dtype: float64,
 'Season 2': 0       1.394211
 1       0.008122
 2       0.522321
 3      14.208346
 4       6.446175
          ...    
 430     0.000253
 431     0.185126
 432    11.956538
 433    24.103849
 434     4.184581
 Length: 435, dtype: float64,
 'Season 3': 0      0.598678
 1      0.236500
 2      0.065712
 3      3.131176
 4      7.962099
          ...   
 435    0.041253
 436    0.392341
 437    0.768648
 438    0.847491
 439    0.482104
 Length: 440, dtype: float64,
 'Season 4': 0       1.610382
 1       0.259350
 2      15.436444
 3       4.662345
 4       0.001535
          ...    
 435     0.098446
 436     1.529883
 437     2.042093
 438     2.801597
 439     1.748107
 Length: 440, dtype: float64}